# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [49]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [50]:
# Hacé la separación en esta celda
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [51]:
# Creá en esta celda la variable param_grid
param_grid = [{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}]

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [52]:
# Importa y crea un GridSearchCV en esta celda
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
grid_search = GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [53]:
# Hace el fit de grid search en esta celda
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [54]:
grid_search.scorer_

make_scorer(mean_squared_error, greater_is_better=False)

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [55]:
# Mostrá los grid_scores en esta celda
np.sqrt(-grid_search.cv_results_['mean_train_score'])

array([31252.94396592, 30984.23279611, 31133.58449174, 30471.73820031,
       30882.65667817, 30766.71470611, 31094.28983789, 30442.13000726,
       30322.025255  , 29093.94167784, 29300.10962411, 30661.00206033,
       29479.95961705, 29284.10446308, 27543.763765  , 30793.89007811,
       29670.05168027, 28814.61735631, 27658.44915785, 27668.78319755,
       30009.88694235, 29885.71505295, 28779.66702669, 27221.97553523,
       27843.52574329])

In [56]:
np.sqrt(-grid_search.cv_results_['mean_test_score'])

array([31256.32439194, 31014.27240363, 31212.49610075, 30399.09563782,
       30889.30749529, 30993.25395333, 31206.88004782, 30404.21477163,
       30381.25159996, 29075.30812231, 29568.17836856, 30854.61353631,
       29809.29198187, 29241.54042241, 27476.89008617, 31099.25895816,
       30124.03584944, 29400.68628448, 28026.38621049, 27941.07760125,
       30203.83003952, 30051.88364822, 28863.0722775 , 27500.16348827,
       28319.43733116])

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [57]:
# Mostrás los resultados en esta celda
grid_search.best_params_

{'max_depth': 3, 'max_features': 5}

In [58]:
grid_search.best_score_

-754979488.8076588

Convertimos a RMSE.

In [59]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

print("Promedio GridSearch RMSE: ", np.round(np.sqrt(-grid_search.cv_results_['mean_test_score']).mean()))

print("Mejor Score RMSE: ", nmsq2rmse(grid_search.best_score_))

Promedio GridSearch RMSE:  29813.0
Mejor Score RMSE:  27476.89


__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

In [60]:
param_grid = [{'min_samples_split': [2, 10, 20], 'max_depth': [None, 2, 5, 10, 15], \
              'min_samples_leaf': [1, 5, 10, 15], 'max_leaf_nodes': [None, 5, 10, 20]}]
tree_reg = DecisionTreeRegressor()
grid_search = GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'min_samples_split': [2, 10, 20], 'max_depth': [None, 2, 5, 10, 15], 'min_samples_leaf': [1, 5, 10, 15], 'max_leaf_nodes': [None, 5, 10, 20]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [61]:
## extracto de mean_train_scores
np.sqrt(-grid_search.cv_results_['mean_train_score'])[:10]

array([  465.63809915, 10775.97923568, 14114.66482189, 13828.7705609 ,
       13826.68367857, 15612.8544353 , 16670.80453007, 16671.27216186,
       16671.27216186, 17879.82596972])

In [62]:
## extracto de mean_test_scores
np.sqrt(-grid_search.cv_results_['mean_test_score'])[:10]

array([26173.46641484, 24335.41486432, 23102.79464655, 22959.17050811,
       23005.39227739, 22267.88894368, 21874.19298765, 21899.62317451,
       21884.69726338, 21577.12148868])

In [63]:
grid_search.best_params_

{'max_depth': 10,
 'max_leaf_nodes': None,
 'min_samples_leaf': 15,
 'min_samples_split': 2}

In [64]:
grid_search.best_score_

-459407773.74392384

In [65]:
print("Promedio GridSearch RMSE: ", np.round(np.sqrt(-grid_search.cv_results_['mean_test_score']).mean()))

print("Mejor Score RMSE: ", nmsq2rmse(grid_search.best_score_))

Promedio GridSearch RMSE:  23201.0
Mejor Score RMSE:  21433.8


Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [66]:
optimised_decision_tree = grid_search.best_estimator_

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [67]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

21301.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [68]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [69]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [70]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798
